In [72]:
# 1. import libraries needed
import os
import psycopg2
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
load_dotenv()


True

In [73]:
#2 read csv files into pandas dataframes
movies = pd.read_csv(r"C:\Users\Darko\DataIngestion\tables\movies.csv")
links = pd.read_csv(r"C:\Users\Darko\DataIngestion\tables\links.csv")
ratings = pd.read_csv(r"C:\Users\Darko\DataIngestion\tables\ratings.csv")
tags = pd.read_csv(r"C:\Users\Darko\DataIngestion\tables\tags.csv")


In [74]:
#2.1 Transform files
#convert timestamp column to numeric then convert to datetime
tags["timestamp"] = pd.to_numeric(tags["timestamp"], errors='coerce')  # Convert to numeric
tags["timestamp"] = pd.to_datetime(tags["timestamp"], unit='s')  # Convert to datetime
print((tags["timestamp"].head()))

ratings["timestamp"] = pd.to_datetime(ratings["timestamp"],unit = 's')
print((ratings["timestamp"].head()))




0   2015-10-24 19:29:54
1   2015-10-24 19:29:56
2   2015-10-24 19:29:52
3   2015-10-24 19:33:27
4   2015-10-24 19:33:20
Name: timestamp, dtype: datetime64[ns]
0   2016-11-19 08:08:20
1   2017-05-03 22:25:45
2   2016-11-19 07:56:11
3   2016-11-19 08:15:31
4   2016-11-19 08:57:33
Name: timestamp, dtype: datetime64[ns]


In [75]:
#3 call environment variables for security
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")
db_port = os.getenv("DB_PORT")

In [76]:
# Create Query Engine
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [ ]:
#Create Function to insert data into sql for all dataframes
import sqlalchemy


def insert_tables(**kwargs):
    for df_name,df in kwargs.items():
        #if block to force postgres to keep timestamp columns as timestamp
        if df_name in ('tag','rating'):
            df.to_sql(
                name = df_name,
                con = engine,
                if_exists = 'replace',
                index = False,
                dtype = {'timestamp':sqlalchemy.types.TIMESTAMP}
            )
            
        else:
            df.to_sql(
                name = df_name,
                con = engine,
                if_exists = 'replace',
                index = False
            )   

In [ ]:
#Connect to the database 
with engine.connect() as con:
    #call function and pass dataframes and table names as keyword arguments with tablenames as keys
    try:
        insert_tables(tag=tags,movie=movies,rating=ratings,link=links)
    except sqlalchemy.exc.SQLAlchemyError as e:
        print(e)
    else:
        print(f'Successfully created/inserted tables into {db_name}')

Successfully created/inserted tables into postgres
